<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/bangla_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import csv
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from nltk import word_tokenize

In [31]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\subor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
import json
with open('sdb.json', encoding='utf-8') as f:
    datastore = json.load(f)
    
sentences = []
labels = []

for item in datastore:
    sentences.append(item['Text'])
    labels.append(item['Label'])

In [33]:
datastore[0]

{'Text': 'অনেকে কাশ্মিরকে পৃথিবীর ভূসর্গ বলেছেন, আবার অনেকে সুইজারল্যান্ডের চাইতেও সুন্দর বলেছেন',
 'Reason': '',
 'Label': 'positive',
 'Source of Data': 'FP',
 'Date': '08/11/2020'}

In [34]:
len(sentences)

968

In [35]:
set(labels)

{'negative', 'neutral', 'positive'}

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
le=LabelEncoder()
label=le.fit_transform(labels)

In [38]:
label[0]

2

In [39]:
le.classes_

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [40]:
sentences

['অনেকে কাশ্মিরকে পৃথিবীর ভূসর্গ বলেছেন, আবার অনেকে সুইজারল্যান্ডের চাইতেও সুন্দর বলেছেন',
 'আমরাও উঠে গেলাম আর বরফ নিয়ে খেলা করতে শুরু করলাম। স্নিগ্ধ-বাতাস মনের কোঠায় এক প্রশান্তি এনে দিল।',
 'সাজেক এর রাস্তা যথেষ্ট ভালো',
 'যে বান্দরবন এর থানচি দেখে নি , সে বাংলাদেশ এর সৌন্দর্য দেখে নি',
 'মারায়ংতং চূড়া হাজার গুণ সুন্দর এবং ন্যাচারাল',
 'মেঘ ছাড়া সাজেকের পুরোটাই কৃত্রিম',
 'হিমালয় কন্যা কাঞ্চন জঙ্ঘা 😍😍 এ যেন এক অপরুপ সৌন্দর্য ',
 'অপরূপ সৌন্দর্য ভরা আমাদের পঞ্চগড়',
 'একদম পচা জায়গা এইডার জন্য আবার টাকা দিয়া ভিডিও করতে হবে?',
 'এইখানে কয়দিন পরে কেউ যাইবো না। একে তো ভাংগা ওয়াচ টাওয়ার, উঠতে দেয় না,নৌকার সিন্ডিকেট,সিএনজির সিন্ডিকেট হাবিজাবি। আজাইরা কাহিনি ।',
 'কয়দিন পর লোকজন ওখানে হিসু করতেও যাবে না',
 'সব বাইকারের স্বপ্নের রোড লাদাখ❤️',
 'লাদাখ এক টুকরো স্বর্গের নাম',
 'লাদাখের আবহাওয়া আমার সবচেয়ে বেশি ভালো লেগেছে',
 'সাজেক কে আমার কাছের ওভাররেটেড মনে হয়েছে',
 'বান্দরবানে হাজার টা সাজেক আছে ভাই।সাজেকের বাপ-দাদা ও বান্দরবানে আছে😁',
 'বাংলাদেশের অ্যাডভেঞ্চার মানেই বান্দরবান',
 'সাজ

In [41]:
def read_stopwords(stopwords_file):
    stop_words = []
    with open(stopwords_file, encoding='utf-8-sig') as f:
        for line in f:
            stop_words.append(line.strip())
    return stop_words

In [42]:
stop_words = read_stopwords('stopwords-bn.txt')
stop_words

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [43]:
!pip install git+https://github.com/banglakit/lemmatizer.git#egg=banglakit-lemmatizer

  Cloning https://github.com/banglakit/lemmatizer.git to c:\users\subor\appdata\local\temp\pip-install-09thtn3_\banglakit-lemmatizer_3e7a4027aeaa479b84f707254c6b08e3


  Running command git clone -q https://github.com/banglakit/lemmatizer.git 'C:\Users\subor\AppData\Local\Temp\pip-install-09thtn3_\banglakit-lemmatizer_3e7a4027aeaa479b84f707254c6b08e3'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/banglakit/lemmatizer.git 'C:\Users\subor\AppData\Local\Temp\pip-install-09thtn3_\banglakit-lemmatizer_3e7a4027aeaa479b84f707254c6b08e3'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [44]:
!pip install banglakit
from banglakit import lemmatizer as lem
from banglakit.lemmatizer import BengaliLemmatizer

lemmatizer = BengaliLemmatizer()

ERROR: Could not find a version that satisfies the requirement banglakit
ERROR: No matching distribution found for banglakit


ModuleNotFoundError: No module named 'banglakit'

In [ ]:
sent=[]
for sen in sentences:
  words = word_tokenize(sen)
  word = [lemmatizer.lemmatize(i, pos=lem.POS_PROPN) for i in words if i not in stop_words]
  sen = ' '.join(word)
  sent.append(sen)

In [ ]:
sent[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(max_features=1000)

In [ ]:
features=cv.fit_transform(sent)

In [ ]:
features=features.toarray()

In [ ]:
features

In [ ]:
len(cv.get_feature_names())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
feature_train,feature_test,label_train,label_test=train_test_split(features,label,test_size=0.2,random_state=7)

In [ ]:
model=MultinomialNB()
model.fit(feature_train,label_train)

In [ ]:
label_pred=model.predict(feature_test)

In [ ]:
label_pred

In [ ]:
label_test

In [ ]:
import sklearn.metrics as m

In [ ]:
m.accuracy_score(label_test,label_pred)

In [ ]:
print(m.classification_report(label_test,label_pred))

In [ ]:
print(m.confusion_matrix(label_test,label_pred))